In [1]:
from google.colab import drive
drive.mount('/drive/')
import os
os.chdir('/drive/My Drive/SKRA/NLP')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive/


In [0]:
import os
os.chdir('/drive/My Drive/SKRA/NLP')

In [4]:
ls

chatbot-countvectorizer-cosine.ipynb  chat_bot.csv  TextRanking.ipynb


In [11]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize # tokenization
from nltk.stem import WordNetLemmatizer # Lemmatization
from nltk import pos_tag # pos tagging
#from nltk.stem import PorterStemmer # stemming
from nltk.corpus import wordnet # wordnet
import re # regular expression
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.remove('what')
stop.remove('which')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's'

In [12]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
chatbot = pd.read_csv('chat_bot.csv',encoding='latin-1')
chatbot.head()

,Questions,Answers
0,What are the prerequisites for this Hadoop Tra...,There are no prerequisites for learning this c...
1,Do I need to know anything before leaning the ...,There are no prerequisites for learning this c...
2,Do I need to have some programming knowledge t...,There are no prerequisites for learning this c...
3,Is it mandatory to know some kind of programmi...,There are no prerequisites for learning this c...
4,Is programming important to learn Hadoop?,There are no prerequisites for learning this c...


In [0]:
def postag(pos):
  if pos.startswith('N'):
      wp = wordnet.NOUN
  elif pos.startswith('V'):
    wp = wordnet.VERB
  elif pos.startswith('R'):
    wp = wordnet.ADV
  elif pos.startswith('J'):
    wp = wordnet.ADJ
  else:
    wp = wordnet.NOUN
    
  return wp

wnl = WordNetLemmatizer() # intilize wordnetlemmatizer

def texprocess(doc):

  # step-1: lower the text
  doc = doc.lower()
  # step-2: remove special characters
  doc = re.sub(r'[^a-z]',' ',doc)
  # step-3: pos tagging (parts of speech) 
  token = word_tokenize(doc) # tokenization - get the words
  token_pos = pos_tag(token) # tagging parts of speech
  # step-4: stemming
  #ps = PorterStemmer()
  #stemming = [ps.stem(word) for word in token]
  # step-4 : lemma and remove stopwords
  lemma = [wnl.lemmatize(word,pos=postag(pos)) for word,pos in token_pos if word not in stop]

  clean = " ".join(lemma)
  return clean

def cosine(a,b):
  moda = np.linalg.norm(a) # magnitude of a
  modb = np.linalg.norm(b) # magnitude of b
  dotprod = np.dot(a,b) # dot product of vector a and vector b
  # a[0] , b[0] -> remove shape in it , we don't want vector to have some shape
  # i.e, neither column matrix nor row matrix
  cos = dotprod/(moda*modb)
  # print('INFO: similarity between document a and b is =',cos_theta)
  return cos

# Word Embedding
- Count Vectorizer

### Ranking Documents
- cosine similarity

 $cos(a,b) = \frac{\bar a. \bar b}{|a||b|}$

In [0]:
documents = list(chatbot['Questions'])
# Step-1: Text processing
documents = [texprocess(doc) for doc in documents] # text processing of the all the documents

Step-2 : Word Embedding

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X = cv.fit_transform(documents).toarray() # word embedding count vectorizer
print('INFO: shape of array =',X.shape)
print('INFO: Features list =',cv.get_feature_names())
print('INFO: length of features =',len(cv.get_feature_names()))

INFO: shape of array = (726, 484)
INFO: Features list = ['able', 'accept', 'access', 'accredit', 'achive', 'acronym', 'actually', 'advantage', 'afternoon', 'agile', 'agility', 'ai', 'algorithm', 'amason', 'amazon', 'analysis', 'analyst', 'analytics', 'anything', 'anywahre', 'apace', 'apache', 'application', 'apply', 'approach', 'approch', 'architech', 'architect', 'article', 'artificial', 'assistance', 'associate', 'attend', 'automation', 'available', 'average', 'aws', 'back', 'background', 'backup', 'become', 'behind', 'benefit', 'benificear', 'benifits', 'best', 'big', 'bigdata', 'blog', 'blue', 'body', 'bot', 'branch', 'break', 'buesness', 'build', 'building', 'bulk', 'bye', 'call', 'cancel', 'candidate', 'capstone', 'card', 'care', 'career', 'case', 'cd', 'certifaction', 'certificate', 'certification', 'certified', 'certify', 'chalenges', 'challenge', 'chef', 'ci', 'ciao', 'class', 'classification', 'classroom', 'cleaning', 'cloud', 'cod', 'come', 'common', 'company', 'complete', '

### Finding Similar documents

In [0]:
import operator

In [0]:
def chatanswers(query):

  # step-1: text processing
  clean = texprocess(query)
  # step-2: word embedding (count vectorizer)
  b = cv.transform([query]).toarray() # query in list

  cosvalue ={}
  for i,vector in enumerate(X):
    cos = cosine(vector,b[0]) # b[0] -> remove shape in it 
    cosvalue.update({i:cos}) # append values in dictonary

  #df['cos'] = cosvalue.values()
  #df.sort_values(by='cos',ascending=False)
  sort = sorted(cosvalue.items(), key=operator.itemgetter(1),reverse=True)
  ind = [index for index,cosv in sort[:5]][0]
  return ind,str(chatbot.loc[ind]['Answers'])

In [47]:
query = 'what is data science ?'
index, ans = chatanswers(query)
print(ans)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide


In [51]:
while True:

  chatinput = input('Srikanth: ')
  if chatinput == 'exit':
    print('Thank you very much have a nice day !!!')
    break
    
  ind, ans = chatanswers(chatinput)
  print(ans)

Srikanth: exit
Thank you very much have a nice day !!!
